In [1]:
import mysql.connector
import random
from faker import Faker

In [2]:
conn = mysql.connector.connect(
    host = '127.0.0.1',
    user = 'root',
    password = 'Sahi@123',
    database='zomdb'
)

In [3]:
fake = Faker()


In [4]:
cursor = conn.cursor()

# Customers Table

In [5]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    email VARCHAR(255),
    phone VARCHAR(20),
    location VARCHAR(255),
    signup_date DATE,
    is_premium BOOLEAN,
    preferred_cuisine VARCHAR(50),
    total_orders INT,
    average_rating FLOAT
)
''')

In [6]:
cuisines = ['Indian', 'Chinese', 'Mexican', 'Italian', 'Japanese', 'Thai']

for _ in range(100):  # Insert 100 customers
    cursor.execute('''
    INSERT INTO Customers (name, email, phone, location, signup_date, is_premium, preferred_cuisine, total_orders, average_rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        fake.name(),
        fake.email(),
        fake.phone_number()[:12],
        fake.address(),
        fake.date_this_decade(),
        random.choice([True, False]),
        random.choice(cuisines),
        random.randint(1, 100),
        round(random.uniform(1, 5), 2)
    ))
conn.commit()

# Restaurants Table

In [7]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Restaurants (
    restaurant_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    cuisine_type VARCHAR(50),
    location VARCHAR(255),
    owner_name VARCHAR(255),
    average_delivery_time INT,
    contact_number VARCHAR(20),
    rating FLOAT,
    total_orders INT,
    is_active BOOLEAN
)
''')

In [8]:
for _ in range(30):  # Insert 30 restaurants
    cursor.execute('''
    INSERT INTO Restaurants (name, cuisine_type, location, owner_name, average_delivery_time, contact_number, rating, total_orders, is_active)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        fake.company(),
        random.choice(cuisines),
        fake.address(),
        fake.name(),
        random.randint(20, 60),
        fake.phone_number()[:12],
        round(random.uniform(2, 5), 2),
        random.randint(50, 500),
        random.choice([True, False])
    ))
conn.commit()


# Delivery Persons Table

In [9]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS DeliveryPersons (
    delivery_person_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    contact_number VARCHAR(20),
    vehicle_type VARCHAR(20),
    total_deliveries INT,
    average_rating FLOAT,
    location VARCHAR(255)
)
''')

In [10]:
vehicle_types = ['Bike', 'Car', 'Scooter', 'Bicycle']

for _ in range(50):  # Insert 50 delivery persons
    cursor.execute('''
    INSERT INTO DeliveryPersons (name, contact_number, vehicle_type, total_deliveries, average_rating, location)
    VALUES (%s, %s, %s, %s, %s, %s)
    ''', (
        fake.name(),
        fake.phone_number()[:12],
        random.choice(vehicle_types),
        random.randint(100, 1000),
        round(random.uniform(3, 5), 2),
        fake.city()
    ))
conn.commit()



# Orders Table

In [11]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    restaurant_id INT,
    order_date DATETIME,
    delivery_time DATETIME,
    status VARCHAR(50),
    total_amount FLOAT,
    payment_mode VARCHAR(50),
    discount_applied FLOAT,
    feedback_rating FLOAT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY (restaurant_id) REFERENCES Restaurants(restaurant_id)
)
''')

In [12]:
from datetime import timedelta

data = []
for _ in range(200):
    customer_id = random.randint(1, 100)
    restaurant_id = random.randint(1, 30)
    order_date = fake.date_time_this_year()
    delivery_time = order_date + timedelta(minutes=random.randint(20, 90))

    data.append((
        customer_id,
        restaurant_id,
        order_date,
        delivery_time,
        random.choice(['Pending', 'Delivered', 'Cancelled']),
        round(random.uniform(100, 2000), 2),
        random.choice(['Credit Card', 'Cash', 'UPI']),
        round(random.uniform(0, 500), 2),
        round(random.uniform(1, 5), 2)
    ))



In [13]:
cursor.executemany('''
    INSERT INTO Orders (customer_id, restaurant_id, order_date, delivery_time, status, total_amount, payment_mode, discount_applied, feedback_rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
''', data)

conn.commit()

# Deliveries Table

In [14]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Deliveries (
    delivery_id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT,
    delivery_person_id INT,
    delivery_status VARCHAR(50),
    distance FLOAT,
    delivery_time INT,
    estimated_time INT,
    delivery_fee FLOAT,
    vehicle_type VARCHAR(20),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (delivery_person_id) REFERENCES DeliveryPersons(delivery_person_id)
)
''')

In [ ]:
for _ in range(180):  # Insert 180 deliveries
    order_id = random.randint(1, 200)
    delivery_person_id = random.randint(1, 50)

    cursor.execute('''
    INSERT INTO Deliveries (order_id, delivery_person_id, delivery_status, distance, delivery_time, estimated_time, delivery_fee, vehicle_type)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        order_id,
        delivery_person_id,
        random.choice(['On the way', 'Delivered']),
        round(random.uniform(1, 15), 2),
        random.randint(20, 90),
        random.randint(20, 70),
        round(random.uniform(20, 100), 2),
        random.choice(vehicle_types)
    ))
conn.commit()
